In [1]:
BUCKET_NAME='test-xwh01'
OBS_BASE_PATH=BUCKET_NAME

from modelarts.session import Session
session = Session()

session.download_data(bucket_path="test-xwh01/test-machine/ratings.dat", path="./ratings.dat")
session.download_data(bucket_path="test-xwh01/test-machine/users.dat", path="./users.dat")
session.download_data(bucket_path="test-xwh01/test-machine/movies.dat", path="./movies.dat")
# import same usefull libraries
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
movie_rating = pd.read_csv('./ratings.dat')

Successfully download file test-xwh01/test-machine/ratings.dat from OBS to local ./ratings.dat
Successfully download file test-xwh01/test-machine/users.dat from OBS to local ./users.dat
Successfully download file test-xwh01/test-machine/movies.dat from OBS to local ./movies.dat


In [2]:
movie_rating_cols = ['user_id',  'movie_id', 'rating', 'unix_tiemstamp'] 
movie_rating = pd.read_csv('./ratings.dat',sep = '::', names = movie_rating_cols, parse_dates = True)
movie_rating.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,user_id,movie_id,rating,unix_tiemstamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
movie_rating.nunique()

user_id             6040
movie_id            3706
rating                 5
unix_tiemstamp    458455
dtype: int64

In [4]:
user=pd.read_csv('./users.dat',sep='::',names=['UserID','Gender','Age','Occupation','Zip-code'])
user.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
user.shape

(6040, 5)

In [6]:
movie=pd.read_csv('./movies.dat',sep='::',names=['MovieID','Title','Genres'])
movie.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movie.shape

(3883, 3)

In [8]:
movie_rating.drop("unix_tiemstamp",inplace = True, axis = 1)
movie_rating.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [9]:
# 创建用户电影矩阵
movie_rating_matrix = movie_rating.pivot_table(index = ['movie_id'], columns = ['user_id'],
                                              values = 'rating').reset_index(drop = True)
movie_rating_matrix.fillna(0, inplace = True)
cmu = movie_rating_matrix
cmu.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 利用欧几里得算法获得电影相似度矩阵
movie_similarity = pairwise_distances(movie_rating_matrix.values, metric = "euclidean")
np.fill_diagonal(movie_similarity,0)
# 将相似度填回相似矩阵
movie_rating_matrix = pd.DataFrame(movie_similarity)
# 欧几里得归一化,使之落在01区间
movie_rating_matrix_sim = (movie_rating_matrix-movie_rating_matrix.min()) / (movie_rating_matrix.max() - movie_rating_matrix.min())
movie_rating_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,0.000000,178.131974,186.780620,189.860475,186.550261,189.185095,184.816125,191.415778,192.015624,182.249829,...,192.426609,192.924856,192.304446,192.512337,193.602686,190.428464,193.292524,192.351241,193.532943,194.170029
1,178.131974,0.000000,98.833193,90.851527,91.782351,132.332914,100.439036,87.355595,88.774996,110.661646,...,92.460803,88.853813,88.555068,89.788641,90.779954,126.261633,107.112091,90.620086,91.618775,109.922700
2,186.780620,98.833193,0.000000,72.897188,74.337070,129.065875,86.936759,72.394751,72.187256,113.991228,...,75.940766,69.892775,70.398864,72.235725,73.776690,119.256027,96.286032,73.742796,74.256313,98.655968
3,189.860475,90.851527,72.897188,0.000000,57.428216,123.450395,77.058419,44.955534,46.249324,110.326787,...,48.445846,38.275318,39.724048,44.922155,46.119410,112.782977,79.730797,46.647615,45.210618,82.286086
4,186.550261,91.782351,74.337070,57.428216,0.000000,126.364552,78.866977,58.249464,58.146367,109.526253,...,62.729578,54.525224,55.371473,59.008474,60.638272,114.682169,87.732548,60.099917,60.066630,89.022469


In [12]:
movie_rating_matrix_sim.head()

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,0.000000,0.710247,0.743562,0.742164,0.733677,0.782412,0.733686,0.740095,0.744532,0.737437,...,0.748742,0.744939,0.742926,0.747172,0.749189,0.785087,0.772991,0.746614,0.749644,0.775039
1,0.767319,0.000000,0.393449,0.355138,0.360968,0.547288,0.398725,0.337754,0.344221,0.447770,...,0.359770,0.343090,0.342113,0.348484,0.351293,0.520544,0.428349,0.351743,0.354882,0.438762
2,0.804573,0.394067,0.000000,0.284955,0.292358,0.533777,0.345123,0.279909,0.279903,0.461242,...,0.295489,0.269876,0.271970,0.280359,0.285495,0.491661,0.385055,0.286234,0.287629,0.393790
3,0.817840,0.362243,0.290199,0.000000,0.225857,0.510553,0.305908,0.173817,0.179330,0.446415,...,0.188505,0.147792,0.153465,0.174350,0.178469,0.464975,0.318849,0.181063,0.175122,0.328449
4,0.803581,0.365954,0.295931,0.224487,0.000000,0.522605,0.313087,0.225217,0.225460,0.443176,...,0.244084,0.210538,0.213915,0.229021,0.234653,0.472805,0.350849,0.233279,0.232666,0.355337


In [13]:
# 获得TOP-N推荐
user_inp = "Toy Story (1995)"
inp = movie[movie['Title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = movie_rating_matrix_sim.iloc[inp]
movie.columns = ['MovieID','Title','Genres','similarity']
movie.head()

,MovieID,Title,Genres,similarity
0,1,Toy Story (1995),Animation|Children's|Comedy,0.000000
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.710247
2,3,Grumpier Old Men (1995),Comedy|Romance,0.743562
3,4,Waiting to Exhale (1995),Comedy|Drama,0.742164
4,5,Father of the Bride Part II (1995),Comedy,0.733677
